In [4]:
# Download Libraries
%pip install google-cloud-storage
%pip install pyarrow # Apache Arrow
%pip install boto3
%pip install pandas psycopg2 
%pip install --upgrade psycopg2
%pip install sqlalchemy-redshift


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Using cached sqlalchemy_redshift-0.8.14-py2.py3-none-any.whl (38 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import io
from io import StringIO
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
import os
import datetime
#from google.cloud import bigquery
import unicodedata
from sqlalchemy import create_engine


In [6]:
#Gathering Data
#Extracting from datasource to view the head 

URL = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Define the data type for column 17 because it contains mixed data types including NaN
dtype_mapping = {'Exam No': 'object', 'List No': 'object', 'List Title Code': 'object', 'Group No': 'object', 'List Agency Code': 'object', 'List Div Code': 'object', 'Veteran Credit': 'object'}


# Specify NaN values to be considered as missing values
na_values = ['NaN', '', 'NA', 'nan']

df_raw = pd.read_csv(URL, dtype=dtype_mapping, na_values=na_values, low_memory=False)

print(df_raw.info())
print('\n')
print(df_raw.shape)
print('\n')
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491645 entries, 0 to 491644
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Exam No             491645 non-null  object 
 1   List No             491645 non-null  object 
 2   First Name          491638 non-null  object 
 3   MI                  272377 non-null  object 
 4   Last Name           491637 non-null  object 
 5   Adj. FA             491645 non-null  float64
 6   List Title Code     491645 non-null  object 
 7   List Title Desc     491645 non-null  object 
 8   Group No            491645 non-null  object 
 9   List Agency Code    491645 non-null  object 
 10  List Agency Desc    491645 non-null  object 
 11  List Div Code       0 non-null       object 
 12  Published Date      221816 non-null  object 
 13  Established Date    476876 non-null  object 
 14  Anniversary Date    476876 non-null  object 
 15  Extension Date      222939 non-nul

,Exam No,List No,First Name,MI,Last Name,Adj. FA,List Title Code,List Title Desc,Group No,List Agency Code,List Agency Desc,List Div Code,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit
0,7001,17897.000,MACDANIEL,NaN,CHARLES,94.00,70310,FIREFIGHTER,000,000,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,Residency Credit
1,3055,52.000,MARIUSZ,NaN,PRZEZDZIECKI,85.85,92005,CARPENTER,000,000,OPEN COMPETITIVE,NaN,04/10/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1111,634.000,TERRENCE,NaN,HAYES,70.00,13652,CERTIFIED IT ADMINISTRATOR (LAN/WAN),000,000,OPEN COMPETITIVE,NaN,05/18/2022,04/19/2023,04/19/2027,NaN,NaN,NaN,NaN,NaN
3,7001,21755.000,DANIEL,NaN,MELENDEZ,93.00,70310,FIREFIGHTER,000,000,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,Residency Credit
4,7001,29676.000,KAREEM,P,MYERS,89.00,70310,FIREFIGHTER,000,000,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,NaN


In [22]:
#Storing Data
# Specify the path to your service account key file
#service_account_key_file = '/Users/karmayangchentenzin/Downloads/Service_Key_HW.json'
service_account_key_file = 'service_acc_key.json' 

# Optionally, load other configuration settings from the JSON file
with open(service_account_key_file, 'r') as f:
    config = json.load(f)

# Fetch data from the web (CSV file)
url = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Read the contents of the CSV file
response = requests.get(url)

# Check if the request was successful
if response.status_code != 200:
    print("Failed to fetch CSV file.")
    exit()

# Read the CSV data into a DataFrame
try: 
    df = pd.read_csv(io.StringIO(response.text), dtype=dtype_mapping, na_values=na_values, low_memory=False)

    # Add timestamp column
    df['load_date'] = datetime.datetime.now() #Date the data is stored. A new column named "load_data" is created.

    # Replace periods with underscores in column names #This had to be done before the original data was loaded from the source to GC because when creating tables in BigQuery, an error message was shown saying that field names can not contain a period (.)
    df.columns = df.columns.str.replace('.', '_')

    # Convert DataFrame to PyArrow Table
    table = pa.Table.from_pandas(df)

    # Write PyArrow Table to Parquet format
    parquet_file_name = 'ny_civil_service_exam.parquet'  # name for the Parquet file
    pq.write_table(table, parquet_file_name)

    # Upload Parquet file to Google Cloud Storage
    bucket_name = 'cis4400_hw1_kyt'
    blob_name = parquet_file_name

    client = storage.Client.from_service_account_json(service_account_key_file)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(parquet_file_name)

    print(f"Parquet file '{parquet_file_name}' uploaded to {bucket_name}/{blob_name} in Google Cloud Storage.")
except Exception as e:
    print(f"Error occurred: {e}")

Parquet file 'ny_civil_service_exam.parquet' uploaded to cis4400_hw1_kyt/ny_civil_service_exam.parquet in Google Cloud Storage.


In [10]:
# Extracting the data from Google Cloud

service_account_key_file = 'service_acc_key.json' 

# Optionally, load other configuration settings from the JSON file
with open(service_account_key_file, 'r') as f:
    config = json.load(f)

# Initialize Google Cloud Storage client
client = storage.Client.from_service_account_json(service_account_key_file)

# Specify the bucket name and Parquet file name
bucket_name = 'cis4400_hw1_kyt'
parquet_file_name = 'ny_civil_service_exam.parquet'

# Download Parquet file from Google Cloud Storage
bucket = client.bucket(bucket_name)
blob = bucket.blob(parquet_file_name)
blob.download_to_filename(parquet_file_name)

# Read Parquet file into a PyArrow Table
table = pq.read_table(parquet_file_name)

# Convert PyArrow Table back to DataFrame
parquet_df = table.to_pandas()

# Print the shape of the DataFrame
print("Shape of DataFrame:", parquet_df.shape)

# Make a copy of the DataFrame
df_copy = parquet_df.copy()

Shape of DataFrame: (491645, 21)


In [18]:
df_copy.columns

Index(['Exam No', 'List No', 'First Name', 'MI', 'Last Name', 'Adj_ FA',
       'List Title Code', 'List Title Desc', 'Group No', 'List Agency Code',
       'List Agency Desc', 'List Div Code', 'Published Date',
       'Established Date', 'Anniversary Date', 'Extension Date',
       'Veteran Credit', 'Parent Lgy Credit', 'Sibling Lgy Credit',
       'Residency Credit', 'load_date'],
      dtype='object')

In [19]:
df_copy.head()

,Exam No,List No,First Name,MI,Last Name,Adj_ FA,List Title Code,List Title Desc,Group No,List Agency Code,...,List Div Code,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit,load_date
0,7001,17897.000,MACDANIEL,None,CHARLES,94.00,70310,FIREFIGHTER,000,000,...,None,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-05-02 06:50:53.299549
1,3055,52.000,MARIUSZ,None,PRZEZDZIECKI,85.85,92005,CARPENTER,000,000,...,None,04/10/2024,None,None,None,None,None,None,None,2024-05-02 06:50:53.299549
2,1111,634.000,TERRENCE,None,HAYES,70.00,13652,CERTIFIED IT ADMINISTRATOR (LAN/WAN),000,000,...,None,05/18/2022,04/19/2023,04/19/2027,None,None,None,None,None,2024-05-02 06:50:53.299549
3,7001,21755.000,DANIEL,None,MELENDEZ,93.00,70310,FIREFIGHTER,000,000,...,None,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-05-02 06:50:53.299549
4,7001,29676.000,KAREEM,P,MYERS,89.00,70310,FIREFIGHTER,000,000,...,None,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,None,2024-05-02 06:50:53.299549


In [9]:
#Tranformation 

# transformation code 1
df_copy['List No'] = df_copy['List No'].astype(str).str.replace(r'\..*', '', regex=True).astype(int)

# transformation code 2
# List of date columns to be formatted
date_columns = ['Published Date', 'Established Date', 'Anniversary Date', 'Extension Date']

# Convert date columns to datetime objects and format them
for col in date_columns:
    df_copy[col] = pd.to_datetime(df_copy[col]).dt.strftime('%Y-%m-%d')

#transformation code 3
date_column = 'load_date'  

# Convert the date column to datetime object and format it
df_copy[date_column] = pd.to_datetime(df_copy[date_column]).dt.strftime('%Y-%m-%d %H:%M:%S')

# transformation code 4
df_copy = df_copy.drop(columns=['List No', 'Group No', 'List Agency Code', 'List Div Code'])

#transformation code 5
# Drop rows with null values in the 'Adj_FA' field
df_copy = df_copy.dropna(subset=['Adj_ FA'])

# transformation code 6
# Remove duplicate rows from the DataFrame
df_copy = df_copy.drop_duplicates()

df_copy.head()

,Exam No,First Name,MI,Last Name,Adj_ FA,List Title Code,List Title Desc,List Agency Desc,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit,load_date
0,7001,MACDANIEL,None,CHARLES,94.00,70310,FIREFIGHTER,OPEN COMPETITIVE,2018-06-13,2019-02-27,2023-02-27,2025-02-27,None,None,None,Residency Credit,2024-05-02 06:50:53
1,3055,MARIUSZ,None,PRZEZDZIECKI,85.85,92005,CARPENTER,OPEN COMPETITIVE,2024-04-10,NaN,NaN,NaN,None,None,None,None,2024-05-02 06:50:53
2,1111,TERRENCE,None,HAYES,70.00,13652,CERTIFIED IT ADMINISTRATOR (LAN/WAN),OPEN COMPETITIVE,2022-05-18,2023-04-19,2027-04-19,NaN,None,None,None,None,2024-05-02 06:50:53
3,7001,DANIEL,None,MELENDEZ,93.00,70310,FIREFIGHTER,OPEN COMPETITIVE,2018-06-13,2019-02-27,2023-02-27,2025-02-27,None,None,None,Residency Credit,2024-05-02 06:50:53
4,7001,KAREEM,P,MYERS,89.00,70310,FIREFIGHTER,OPEN COMPETITIVE,2018-06-13,2019-02-27,2023-02-27,2025-02-27,None,None,None,None,2024-05-02 06:50:53


In [37]:
print(df_copy.shape) #This shows that there are some rows that are duplicates because the number or rows differs from the original.

(491536, 17)


In [37]:
redshift_username = 'kyt'
redshift_password = 'Kimmashinzee1'
redshift_host = 'cis4400-hw1-cluster.cuclkm2mtyo1.us-east-2.redshift.amazonaws.com'
redshift_port = '5439'
redshift_db = 'dev'


In [38]:
conn_str = f'postgresql+psycopg2://{redshift_username}:{redshift_password}@{redshift_host}:{redshift_port}/{redshift_db}'


In [39]:
engine = create_engine(conn_str)
connection = engine.connect()


In [44]:
import pandas as pd
from sqlalchemy import create_engine

# Define the connection URL for your Redshift cluster
#redshift_url = 'postgresql+psycopg2://<username>:<password>@<host>:<port>/<database>'

# Create the SQLAlchemy engine
#engine = create_engine(redshift_url)

# Define the table name in your Redshift database
table_name = 'dim_candidate_info'

# Specify the columns you want to load from the DataFrame
columns_to_load = ['First Name', 'MI', 'Last Name']  # Adjust column names as needed

# Insert data into the Redshift table using DataFrame.to_sql
#with engine.connect() as conn:
    #df_copy[columns_to_load].to_sql(name=table_name, con=conn, if_exists='append', index=False)

# Insert data into the Redshift table using connection.execute()
with engine.connect() as conn:
    for index, row in df_copy[columns_to_load].iterrows():
        conn.execute(
            f"INSERT INTO {table_name} (\"first_name\", middle_initial, \"last_name\") "
            "VALUES (%s, %s, %s)",
            (row['First Name'], row['MI'], row['Last Name']))

print("Data loaded successfully into Redshift.")


KeyboardInterrupt: 

In [47]:
import pandas as pd
from sqlalchemy import create_engine

# Define the connection URL for your Redshift cluster
#redshift_url = 'postgresql+psycopg2://<username>:<password>@<host>:<port>/<database>'

# Create the SQLAlchemy engine
#engine = create_engine(redshift_url)

# Define the table name in your Redshift database
table_name = 'dim_candidate_info'

# Specify the columns you want to load from the DataFrame
columns_to_load = ['First Name', 'MI', 'Last Name']  # Adjust column names as needed
# Define batch size for batch inserts
batch_size = 1000

# Insert data into the Redshift table using batch inserts
with engine.connect() as conn:
    for i in range(0, len(df_copy), batch_size):
        batch_df = df_copy.iloc[i:i+batch_size]
        conn.execute(
            f"INSERT INTO {table_name} (\"first_name\", middle_initial, \"last_name\") "
            "VALUES (%s, %s, %s)",
            [tuple(row) for row in batch_df[columns_to_load].values]
            )

print("Data loaded successfully into Redshift.")

KeyboardInterrupt: 

In [41]:


import os

os.environ['SQLALCHEMY_WARN_20'] = '1'


# Specify the columns you want to load from the DataFrame
columns_to_load = ['First Name', 'MI', 'Last Name']  # Adjust column names as needed

# Load your DataFrame
# df = pd.read_csv('your_data.csv')  # Replace 'your_data.csv' with the path to your CSV file

# Select only the columns you want to load
df_selected = df_copy[columns_to_load]

# Define the table name in your Redshift database
table_name = 'dim_candidate_info'  # Replace 'your_table_name' with the name of your Redshift table

# Construct the SQL INSERT statement
# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} (\"first_name\", middle_initial, \"last_name\") VALUES "

# Generate placeholders for parameterized query
placeholders = ', '.join(['%s'] * len(columns_to_load))

# Iterate over rows and execute the INSERT statement
with engine.connect() as conn:
    for row in df_selected.itertuples(index=False, name=None):
        conn.execute(sql + f"({placeholders})", row)

print("Data loaded successfully into Redshift.")



KeyboardInterrupt: 

In [22]:
from sqlalchemy import create_engine
import pandas as pd

# Connection details for Redshift
redshift_username = 'kyt'
redshift_password = 'Group6otc'
#redshift_host = 'cis4400-hw1-cluster.cuclkm2mtyo1.us-east-2.redshift.amazonaws.com'
#redshift_port = '5439'
#redshift_db = 'dev'

# Sample data for demonstration
sample_data = {
    'First Name': ['John', 'Alice', 'Bob'],
    'MI': ['A', 'B', 'C'],
    'Last Name': ['Doe', 'Smith', 'Johnson']
}

# Create a DataFrame from the sample data
df = pd.DataFrame(sample_data)

# Create connection string
conn_str = f'postgresql+psycopg2://{redshift_username}:{redshift_password}@{redshift_host}:{redshift_port}/{redshift_db}'

# Create engine and connect to Redshift
engine = create_engine(conn_str)

# Load DataFrame into Redshift
df.to_sql('dim_candidate_info', engine, index=False, if_exists='replace')

print("Connection established and sample data loaded successfully!")


KeyboardInterrupt: 

In [33]:
# Loading Data into Redshift

# Connection details for Redshift
redshift_username = 'kyt'
redshift_password = 'Kimmashinzee1'
redshift_host = 'cis4400-hw1-cluster.cuclkm2mtyo1.us-east-2.redshift.amazonaws.com'
redshift_port = '5439'
redshift_db = 'dev'

# Define the main DataFrame containing your data
#main_df = df_copy  # main DataFrame containing the data from Google Cloud

# Dictionary mapping table names to column names
table_columns = {
    'dim_candidate_info': ['First Name', 'MI', 'Last Name'],  # Example: table1 contains col1, col2, col3
    #'dim_date': ['', 'col5', 'col6'],  # Example: table2 contains col4, col5, col6
    # Add more tables and their corresponding columns as needed
}

# Function to load DataFrame into existing Redshift table
def load_df_to_redshift(df, table_name):
    # Create connection string
    conn_str = f'postgresql+psycopg2://{redshift_username}:{redshift_password}@{redshift_host}:{redshift_port}/{redshift_db}'
    engine = create_engine(conn_str)

    # Write DataFrame to Redshift (append mode)
    df.to_sql(table_name, engine, index=False, if_exists='append')

if __name__ == '__main__':
    for table_name, columns in table_columns.items():
        # Extract columns for the current table from the main DataFrame
        table_df = df_copy[columns]
        
        # Load the extracted DataFrame into the current Redshift table
        load_df_to_redshift(table_df, table_name)


ProgrammingError: (psycopg2.errors.UndefinedObject) unrecognized configuration parameter "standard_conforming_strings"

[SQL: show standard_conforming_strings]
(Background on this error at: https://sqlalche.me/e/20/f405)